<a href="https://colab.research.google.com/github/reidnersousa/IFB_Compiladores_2023/blob/main/analisador_sintatico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
class SyntaxAnalyzer:
    def __init__(self, lexer):
        self.lexer = lexer
        self.current_token = self.lexer.next_token()

    def eat(self, token_type):
        if self.current_token.type == token_type:
            self.current_token = self.lexer.next_token()
        else:
            raise SyntaxError(f'Expected {token_type}, got {self.current_token.type} instead.')

    def factor(self):
        token = self.current_token
        if token.type == 'LPAREN':
            self.eat('LPAREN')
            expr = self.expr()
            self.eat('RPAREN')
            return expr
        elif token.type == 'INTEGER':
            self.eat('INTEGER')
            return token.value
        else:
            raise SyntaxError(f'Unexpected token {token.type}')

    def term(self):
        result = self.factor()

        while self.current_token.type in ('MULT', 'DIV'):
            op = self.current_token
            if op.type == 'MULT':
                self.eat('MULT')
                result *= self.factor()
            elif op.type == 'DIV':
                self.eat('DIV')
                result /= self.factor()

        return result

    def expr(self):
        result = self.term()

        while self.current_token.type in ('PLUS', 'MINUS'):
            op = self.current_token
            if op.type == 'PLUS':
                self.eat('PLUS')
                result += self.term()
            elif op.type == 'MINUS':
                self.eat('MINUS')
                result -= self.term()

        return result

    def parse(self):
        result = self.expr()
        if self.current_token.type != 'EOF':
            raise SyntaxError('Unexpected token at end of expression')
        return result




In [15]:
class Lexer:
    def __init__(self, text):
        self.text = text
        self.pos = 0

    def next_token(self):
        if self.pos >= len(self.text):
            return Token('EOF', None)
        current_char = self.text[self.pos]

        if current_char.isdigit():
            value = ''
            while self.pos < len(self.text) and self.text[self.pos].isdigit():
                value += self.text[self.pos]
                self.pos += 1
            return Token('INTEGER', int(value))

        if current_char == '+':
            self.pos += 1
            return Token('PLUS', '+')

        if current_char == '-':
            self.pos += 1
            return Token('MINUS', '-')

        if current_char == '*':
            self.pos += 1
            return Token('MULT', '*')

        if current_char == '/':
            self.pos += 1
            return Token('DIV', '/')

        if current_char == '(':
            self.pos += 1
            return Token('LPAREN', '(')

        if current_char == ')':
            self.pos += 1
            return Token('RPAREN', ')')

        # if none of the above match, raise an error
        raise ValueError(f"Invalid character: {current_char}")


In [14]:
class Token:
    def __init__(self, type_, value):
        self.type = type_
        self.value = value

    def __str__(self):
        return f'Token({self.type}, {self.value})'


In [36]:
lexer = Lexer('(12+1)+2*10')
analyzer = SyntaxAnalyzer(lexer)
result = analyzer.parse()
print(result)


33
